## Оценка работы модели microsoft/biogpt в исходном виде

In [40]:
from transformers import AutoTokenizer, BioGptForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
model = BioGptForCausalLM.from_pretrained("microsoft/biogpt")

Попробуем субъективно оценить качество работы модели. Для этого возьмем несколько предложений из медицинских протоколов (и просто из головы). В комментариях ожидаемые ответы

In [41]:
sentences = [
    "The patient has an elevated", # temperature
    "Skin physiological color, moderate", # humidity
    "The tongue is moist, covered with a gray", # coating
    "Complaints of recurrent head", # aches
    "The cervix is ​​located along the wire axis of the", # pelvis
    "Molecular diagnostics of hereditary ", # diseases
    "Infectious diseases: chickenpox, mumps, candidiasis, candidal", # balanitis
    "General cerebral symptoms decreased during", # treatment
    "Moderate age-related dysrhythmia with", # hyperventilation
    "Preliminary diagnosis: Vegetovascular", # dystonia
    "Against the background of signs of cerebral angiodystonia, a moderate increase in blood", # flow
    "Moderately severe vertebrogenic pain", # syndrome
    "CDG with examination of extracranial" # vessels
]

Для каждого предложения выведем три наиболее подходящих варианта

In [43]:
for sent in sentences:
    input_ids = tokenizer.encode(sent, return_tensors="pt")
    outputs = model.generate(input_ids,
                             max_new_tokens=1,
                             do_sample=False,
                             num_beams=5,
                             no_repeat_ngram_size=2,
                             num_return_sequences=3,
                             early_stopping=True,
                             top_k=50)
    i = 0
    for out in outputs:
        i += 1
        print(tokenizer.decode(out, skip_special_tokens=True))
        if i % 3 == 0:
            print("-----------------------------------------")

У пациента наблюдается повышенная температура а
У пациента наблюдается повышенная температура о
У пациента наблюдается повышенная температура с
-----------------------------------------
The patient has an elevated serum
The patient has an elevated level
The patient has an elevated risk
-----------------------------------------
Skin physiological color, moderate-
Skin physiological color, moderate to
Skin physiological color, moderate and
-----------------------------------------
The tongue is moist, covered with a gray,
The tongue is moist, covered with a gray-
The tongue is moist, covered with a gray and
-----------------------------------------
Complaints of recurrent head and
Complaints of recurrent head injury
Complaints of recurrent head trauma
-----------------------------------------
The cervix is located along the wire axis of the applicator
The cervix is located along the wire axis of the probe
The cervix is located along the wire axis of the instrument
-----------------------

Результаты не совсем соответствуют ожидаемым, но это нормально, учитывая прямой перевод предложений с русского на английский через гугл-переводчик. Для получения более корретных результатов желательно использовать англоязычные источники, хотя даже в текущем варианте видно, что модель понимает медицинскую тематику предложений и выдает адекватные результаты